# MySQL Index Skip Scan
## 1. Craete Sample Data

### Data Example

| product_category_id | product_id | created_at |
| --- | --- | --- |
| 1 | 1 | 2019-01-01 |
| 99 | 9999 | 2023-05-31|

In [43]:
# https://stackoverflow.com/questions/553303/generate-a-random-date-between-two-other-dates
import random
import time


def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, "%Y-%m-%d", prop)


print(random_date("2019-01-01", "2023-05-31", random.random()))

2019-04-22


In [44]:
def get_random_date():
    return random_date("2022-01-01", "2023-12-31", random.random())

In [45]:
def get_random_number(max=100):
    return round(random.random() * max)

In [46]:
get_random_number(max=10000)

1216

In [47]:
import pandas as pd


data_cat_100_list = []
data_cat_10000_list = []
for _ in range(1000000):
    data_cat_100_list.append([get_random_number(max=100), get_random_number(max=100000), get_random_date()])
    data_cat_10000_list.append([get_random_number(max=10000), get_random_number(max=100000), get_random_date()])

In [48]:
col_list = ["product_category_id", "product_id", "created_at"]

In [49]:
df_cat_100 = pd.DataFrame(data_cat_100_list, columns=col_list)
df_cat_10000 = pd.DataFrame(data_cat_10000_list, columns=col_list)

In [50]:
df_cat_100.head()

,product_category_id,product_id,created_at
0,32,23002,2023-03-29
1,48,4323,2023-06-29
2,66,72219,2023-01-12
3,16,99460,2023-01-15
4,34,34533,2023-05-30


In [51]:
df_cat_10000.head()

,product_category_id,product_id,created_at
0,9662,98948,2023-05-26
1,1874,95398,2022-11-26
2,6063,10415,2022-01-06
3,7566,78774,2022-11-24
4,5558,55451,2023-09-02


## 2. Create Tables
- product_category_100: 카테고리가 100개
- product_category_10000: 카테고리가 10000개

In [52]:
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

pymysql.install_as_MySQLdb()
# ref: https://stackoverflow.com/questions/4130922/how-to-increment-datetime-by-custom-months-in-python-without-using-library

In [53]:
# User database information
USER = "root"
PASSWORD = "1111"
ENDPOINT = "localhost"

In [54]:
conn = pymysql.connect(
    host=ENDPOINT,
    user=USER,
    password=PASSWORD,
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
)

cur = conn.cursor()

In [55]:
# Create Schema
cur.execute("CREATE DATABASE IF NOT EXISTS real_mysql;")

1

In [56]:
# Create Table

engine = create_engine(f"mysql+mysqldb://{USER}:{PASSWORD}@{ENDPOINT}:3306")
# http://i5on9i.blogspot.com/2020/05/no-module-named-mysqldb.html

df_cat_100.to_sql(
    "product_category_100",
    engine,
    schema="real_mysql",
    index=False,
    if_exists="replace",
    chunksize=10000,
)

df_cat_10000.to_sql(
    "product_category_10000",
    engine,
    schema="real_mysql",
    index=False,
    if_exists="replace",
    chunksize=10000,
)

In [57]:
# Alter Column Type

cur.execute("alter table real_mysql.product_category_100 modify `product_category_id` int;")
cur.execute("alter table real_mysql.product_category_100 modify `product_id` int;")
cur.execute("alter table real_mysql.product_category_100 modify `created_at` date;")
cur.execute("alter table real_mysql.product_category_10000 modify `product_category_id` int;")
cur.execute("alter table real_mysql.product_category_10000 modify `product_id` int;")
cur.execute("alter table real_mysql.product_category_10000 modify `created_at` date;")

1000000

In [58]:
# Create Index (product_category_id, product_id)

idx_product_category_product_id_100 = """
create index idx_product_category_product_id
    on real_mysql.product_category_100 (product_category_id, product_id);
"""

idx_product_category_product_id_1000 = """
create index idx_product_category_product_id
    on real_mysql.product_category_10000 (product_category_id, product_id);
"""

In [59]:
cur.execute(idx_product_category_product_id_100)
cur.execute(idx_product_category_product_id_1000)

0